In [1]:
from mpi4py import MPI

### MPI4Py

In [2]:
%%writefile src/simple_mpi.py

from mpi4py import MPI

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Print "Hello" from all processors
print(f"Hello from process {rank} out of {size}")

# Finalize MPI
MPI.Finalize()

Writing src/simple_mpi.py


MPI execute command (run in commandline)

In [ ]:
!mpiexec -n 4 python src/simple_mpi.py

### Strace wrapper

In [56]:
%%writefile src/sm.sh

#!/bin/bash
LOG_FILE="logs/strace_$$"
strace -f -tt -T -r -y -o "$LOG_FILE" "$@"

Writing src/sm.sh


Non mpi example

In [64]:
!bash src/sm.sh ls

01-create_dataset.ipynb		   08-strace-profiling.ipynb
02-retrieve_dataset.ipynb	   09-simple-mpi.ipynb
03-set_attributes.ipynb		   a.out
04-create_csv_h5_pickle.ipynb	   build
04B-check_sizes_arrays.ipynb	   logs
04C-mmarrays.ipynb		   massif.out.73983
05-discovering_file_structs.ipynb  src
06-hetero-csv-compund.ipynb	   tmp.txt
07-memory-profiling.ipynb


MPI example (should be run on command line)

In [54]:
!mpiexec -n 4 bash src/sm.sh python src/simple_mpi.py 

Redundant wrapper

In [57]:
%%writefile src/smpi.sh

#!/bin/bash
n=$1
shift
mpiexec -n $n bash src/sm.sh "$@"

Overwriting src/smpi.sh


In [58]:
!bash src/smpi.sh 4 python src/simple_mpi.py

### C example

In [59]:
%%writefile src/simple_mpi.c

#include <stdio.h>
#include <mpi.h>

int main(int argc, char** argv) {
    // Initialize MPI
    MPI_Init(&argc, &argv);

    // Get the rank of the current process
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Get the total number of processes
    int size;
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Print "Hello, World!" along with the rank
    printf("Hello, World! I am process %d out of %d.\n", rank, size);

    // Finalize MPI
    MPI_Finalize();

    return 0;
}

Writing src/simple_mpi.c


In [61]:
!mpicc -o src/a.out src/simple_mpi.c

In [63]:
!mpiexec -n 4 bash src/sm.sh ./src/a.out

### MPI IO example in python

In [78]:
%%writefile src/mpi_io.py

from mpi4py import MPI
import numpy as np

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Define the size of the shared numpy array
array_size = 10000  # Adjust the size as needed

# Create a numpy array for each rank
local_data = np.arange(array_size, dtype=np.float64)

# Create a filename for each rank
filename = f"data/data_rank_{rank}.npy"

# Save the local_data to a file
with open(filename,"wb") as f:
    np.save(f, local_data)

# Barrier to synchronize processes before reading
comm.Barrier()


# Finalize MPI
MPI.Finalize()

Overwriting src/mpi_io.py


Simple mpi run

In [79]:
!mpiexec -n 4 python src/mpi_io.py

strace profiling run

In [ ]:
!mpiexec -n 4 bash src/sm.sh python src/mpi_io.py